# Function Dependency Equations for Free Product of Finite Cyclic Groups.

Consider $G=*_{i=1}^k G_i$, $G_i=\langle x_i| x_i^{n_i}=1 \rangle \cong \mathbb{Z}_{n_i}$, where $n_i \in \mathbb{Z}_{\ge2}$. 
Take the generating set $S=\bigcup_{i=1}^k S_i$ for $G$, where 
$S_i=\{x_i,x_i^{-1}\}$ if the `inv_closed` option is set to a `true` value, and $S_i=\{x_i\}$ otherwise.


For $g\in G$ and $X\subseteq G$, let $F_{g,X}(t)$ be series for the class of all words in $S^*$ equivalent to $g\in G$, with proper nonempty prefixes avoiding $X$, characterized by the word length. We set up functional equations involving $F_{g,X}$, for which $\{g\}\cup X\subseteq G_i$ for some $i$. 

Finally, we use Maple solve for $F(t):=F_{1,\emptyset}(t)$ and find it's minimal polynomial .

In [275]:
## Values for user to modify

nis = [3,3] # the n_i values. n_i= nis[i-1].

inv_closed = 1 # do we want the generating set to be inversely closed? Makes no difference if all factors have order 2



In [276]:
## do not change!

k = len(nis)

# Compute free group and gen set
Fk = FreeGroup(k,['x%d'%i for i in (1..k)])

assert(list(map(str,Fk.gens()))==['x%d'%i for i in (1..k)]) # make sure that generators are named xi, i=1,...,k

xs = Fk.gens() # the gens xi=xs[i-1]
rel = []
for v,n in zip(xs,nis):
    rel.append(v^n)
G = Fk/ tuple(rel)

xs = G.gens()


In [277]:
# let's adopt the xi notation, just in case.
for v in xs:
    exec('%s=v'%v)
# now we can type out 'x1' instead of 'xs[0]' for the first generator, etc.
assert(x1==xs[0])

In [278]:
Grs = G.rewriting_system() # a rewriting system to simplfy elements
Grs.make_confluent() # this guarrantees that equivalent elements map to a UNIQUE expression

def recmap(f):
    # f: an single argument function
    def _recmap_f(obj):
        # obj: atomic element or an iterable collection of them
        # return the reduced form for the element via a single argument function f,
        #    of the same iterable type, with each element recursively reduced
        try:
            iter(obj)
        except:
            # not iterable, so must be one atomic element
            return f(obj)
        container = type(obj)
        return container(list(map(_recmap_f, obj)))
    return _recmap_f

rd = recmap(Grs.reduce) # reduce elements of G, or iterable collections of them
Grs

Rewriting system of Finitely presented group < x1, x2 | x1^3, x2^3 >
with rules:
    x1^-2    --->    x1
    x1^2    --->    x1^-1
    x2^-2    --->    x2
    x2^2    --->    x2^-1

In [279]:
Sis = []
S = set()

for i, v in enumerate(xs,1):
    Si = Set([rd(v)])
    if inv_closed:
        Si = Si.union(Set([rd(v^-1)]))
    Si = rd(Si)
    Sis.append(Si)
    S.update(list(Si))
    exec('S%d=Si'%i)
S = Set(S)

In [280]:
Sis,S

([{x1^-1, x1}, {x2, x2^-1}], {x2, x2^-1, x1^-1, x1})

In [290]:
# latex expression for sets
def set_to_latex(A):
    if A == Set():
        return r'\varnothing'
    return latex(A)

In [297]:
# define a space of variables indexed by the (g,X) pairs
# start with what we wish to solve, (1,\emptyset).
def series_expression(g,X):
    return 'F_{%s,%s}(t)'%(latex(g),set_to_latex(X))
var_space = {}
goal_pair = (G.one(), Set())
VAR_COUNTER = 0 # increment every time a new series variable is defined

def add_series_var(pair):
    if pair in var_space:
        return False
    global VAR_COUNTER
    g,X = pair
    var_space[pair] = var('v%d'%VAR_COUNTER, latex_name = series_expression(g,X))
    VAR_COUNTER += 1
    return True

def reset_var_space():
    global VAR_COUNTER
    var_space.clear()
    VAR_COUNTER=0
    add_series_var(goal_pair)
    
reset_var_space()

In [298]:
pretty_print(var_space)

{(1, {}): v0}

In [299]:
# need to define set-set, set-element multiplication
def set_mult(A,B):
    # turn any non-set object into a singleton set containing that object
    # assume elements all belong to G, and written in reduced from
    A,B = map(lambda S: S if S in Sets else Set([S]),[A,B])
    C= Set([a*b for a,b in cartesian_product([A,B])]) # assuming element multiplication is well defined
    return rd(C)

# Let's overload the '*' operator for Set objects
# Set is a function, but returns an instance of some spectifed kind of Set class, which is accessible via .parent()
reverse_if = lambda seq, cond: list(seq) if not cond else list(reversed(seq))
Set().parent()._mul_ =  lambda self, other, switch_sides=False: set_mult(*(reverse_if([self,other], switch_sides)))

In [300]:
# Now get the cyclic factors


def in_cyclic_factor(el, v):
    '''
        returns if the group element is in the given cyclic factor
        el: the element in G to check
        v: one of the xi's, the generator of the factor 
    '''
    expr = el.syllables()
    if not expr:
        return True
    return len(expr)==1 and expr[0][0] == v
    
def subset_of_cyclic_factor(A,v):
    # assume A is a finite sets
    return all((in_cyclic_factor(el,v) for el in A))

In [301]:
var('t')

t

In [302]:
# construct the functional equation given the desired (g,X) pair
# assume {g}\cup X is completely in one of the cyclic factors
# CURRENTLY FINITE CASE ONLY

def get_FD_equation(pair, ret_new_pairs = True):
    '''
        returns a functional depency equation
        if ret_new_pairs is set to True, return a (equation, set of (g,X) pairs) pair instead
        assume g,X are rewritten in simplest form
    '''
    if ret_new_pairs:
        newpairs = set()
    g,X = pair
    eqn = None
    one = G.one()
    oneset = Set([one])
    if one not in X:
        XU1 = X.union(oneset)
        if add_series_var((one,X)) and ret_new_pairs:
            newpairs.add((one,X))
        if add_series_var((g,XU1)) and ret_new_pairs:
            newpairs.add((g,XU1))
        if g==one:
            eqn = 1+var_space[(one,X)]*(var_space[(one,XU1)]-1)
        else:
            eqn = var_space[(one,X)]*var_space[(g,XU1)]
    else:
        for v,gen in zip(xs, Sis):
            if subset_of_cyclic_factor(X,v):
                if g!=one:
                    eqn = t*ZZ(g in gen)
                    for s in gen-X:
                        [gs] = rd(s^-1)*Set([g]) # use set so auto reduction is implemented
                        Xs = rd(s^-1)*X
                        if add_series_var((gs,Xs)) and ret_new_pairs:
                            newpairs.add((gs,Xs))
                        eqn += t*var_space[(gs,Xs)]
                else:
                    eqn = 1*t^0
                    for gen_ext in Sis:
                        if gen_ext==gen:
                            continue
                        for s in gen_ext:
                            sinv = rd(s^-1)
                            set_sinv = Set([sinv])
                            if add_series_var((sinv,set_sinv)) and ret_new_pairs:
                                newpairs.add((sinv,set_sinv))
                            eqn += t*var_space[(sinv,set_sinv)]
                    for s in gen-X:
                        [gs] = rd(s^-1)*Set([g]) # use set so auto reduction is implemented
                        Xs = rd(s^-1)*X
                        if add_series_var((gs,Xs)) and ret_new_pairs:
                            newpairs.add((gs,Xs))
                        eqn += t*var_space[(gs,Xs)]
                
                break
    eqn = var_space[pair]==eqn
    return (eqn,newpairs) if ret_new_pairs else eqn


In [303]:
# Let us build the system of equations
from time import time

reset_var_space()

system = []
queue = [goal_pair]
st_time = time()
# use BFS
while queue:
    pair = queue.pop(0)
    #pretty_print(pair)
    eqn, newp = get_FD_equation(pair)
    queue.extend(newp)
    system.append(eqn)
    end_time =  time()
    if end_time-st_time>100:
        break


In [304]:
# Here are the equations
for eqn in system:
    print(latex(eqn),r'\\')
print()
print()
for eqn in system:
    #print(eqn)
    pretty_print(eqn)

{F_{1,\varnothing}(t)} = {F_{1,\varnothing}(t)} {\left({F_{1,\left\{1\right\}}(t)} - 1\right)} + 1 \\
{F_{1,\left\{1\right\}}(t)} = t {F_{x_{2}^{-1},\left\{x_{2}^{-1}\right\}}(t)} + t {F_{x_{2},\left\{x_{2}\right\}}(t)} + t {F_{x_{1},\left\{x_{1}\right\}}(t)} + t {F_{x_{1}^{-1},\left\{x_{1}^{-1}\right\}}(t)} + 1 \\
{F_{x_{1}^{-1},\left\{x_{1}^{-1}\right\}}(t)} = {F_{1,\left\{x_{1}^{-1}\right\}}(t)} {F_{x_{1}^{-1},\left\{1, x_{1}^{-1}\right\}}(t)} \\
{F_{x_{1},\left\{x_{1}\right\}}(t)} = {F_{1,\left\{x_{1}\right\}}(t)} {F_{x_{1},\left\{1, x_{1}\right\}}(t)} \\
{F_{x_{2},\left\{x_{2}\right\}}(t)} = {F_{1,\left\{x_{2}\right\}}(t)} {F_{x_{2},\left\{x_{2}, 1\right\}}(t)} \\
{F_{x_{2}^{-1},\left\{x_{2}^{-1}\right\}}(t)} = {F_{1,\left\{x_{2}^{-1}\right\}}(t)} {F_{x_{2}^{-1},\left\{1, x_{2}^{-1}\right\}}(t)} \\
{F_{x_{1}^{-1},\left\{1, x_{1}^{-1}\right\}}(t)} = t {F_{x_{1},\left\{x_{1}^{-1}, x_{1}\right\}}(t)} + t \\
{F_{1,\left\{x_{1}^{-1}\right\}}(t)} = {\left({F_{1,\left\{1, x_{1}^{-1}\righ

v0 == v0*(v1 - 1) + 1

v1 == t*v2 + t*v3 + t*v4 + t*v5 + 1

v5 == v6*v7

v4 == v8*v9

...

v25 == v25*(v29 - 1) + 1

v26 == t

v27 == t

v28 == t*v2 + t*v3 + 1

v29 == t*v4 + t*v5 + 1

In [305]:
# There are symmetries amongst the variables such that some variables will repersent the same series.
# Amongst each such equivalence class, we only need to keep one representation.
# To strip symmetry, we need union find.

from collections import defaultdict as DD

class UF(object):
    
    def __init__(self):
        self.V = {}
        self.parent = {}
    
    def add(self, node):
        if node not in self.V:
            self.V[node] = len(self.V)
            self.parent[node]=node
    
    def find(self,u):
        P = self.parent
        while P[u]!=u:
            P[u] = P[P[u]]
            u = P[u]
        return u
    
    def union(self,u,v):
        u,v = map(self.find, [u,v])
        u,v = sorted([u,v],key = lambda s: self.V[s])
        self.parent[v]=u
    
    def verts(self):
        return iter(self.V)
    
    def comps(self):
        ans = DD(set)
        for v in self.V:
            ans[self.find(v)].add(v)
        return ans
    


In [306]:
# initialize the structure to track symmetry
eqcl = UF()
for pair in var_space:
    eqcl.add(pair)

eqcl.comps()

defaultdict(<class 'set'>, {(1, {}): {(1, {})}, (1, {1}): {(1, {1})}, (x2^-1, {x2^-1}): {(x2^-1, {x2^-1})}, (x2, {x2}): {(x2, {x2})}, (x1, {x1}): {(x1, {x1})}, (x1^-1, {x1^-1}): {(x1^-1, {x1^-1})}, (1, {x1^-1}): {(1, {x1^-1})}, (x1^-1, {1, x1^-1}): {(x1^-1, {1, x1^-1})}, (1, {x1}): {(1, {x1})}, (x1, {1, x1}): {(x1, {1, x1})}, (1, {x2}): {(1, {x2})}, (x2, {x2, 1}): {(x2, {x2, 1})}, (1, {x2^-1}): {(1, {x2^-1})}, (x2^-1, {1, x2^-1}): {(x2^-1, {1, x2^-1})}, (x1, {x1^-1, x1}): {(x1, {x1^-1, x1})}, (1, {1, x1^-1}): {(1, {1, x1^-1})}, (1, {1, x1}): {(1, {1, x1})}, (x1^-1, {x1^-1, x1}): {(x1^-1, {x1^-1, x1})}, (x2^-1, {x2, x2^-1}): {(x2^-1, {x2, x2^-1})}, (1, {x2, 1}): {(1, {x2, 1})}, (x2, {x2, x2^-1}): {(x2, {x2, x2^-1})}, (1, {1, x2^-1}): {(1, {1, x2^-1})}, (1, {x1^-1, x1}): {(1, {x1^-1, x1})}, (x1, {1, x1^-1, x1}): {(x1, {1, x1^-1, x1})}, (x1^-1, {1, x1^-1, x1}): {(x1^-1, {1, x1^-1, x1})}, (1, {x2, x2^-1}): {(1, {x2, x2^-1})}, (x2^-1, {x2, 1, x2^-1}): {(x2^-1, {x2, 1, x2^-1})}, (x2, {x2, 1,

In [307]:
# if S is inverse closed, then (g,X) is essentially the same as (g^-1,X^-1)
if inv_closed:
    vis = set()
    for g,X in var_space:
        if (g,X) in vis:
            continue
        ginv = rd(g^-1)
        Xinv = Set([v^-1 for v in X])
        Xinv = rd(Xinv)
        pairinv = (ginv,Xinv)
        if pairinv in var_space:
            vis.add(pairinv)
            eqcl.union((g,X),pairinv)
eqcl.comps()

defaultdict(<class 'set'>, {(1, {}): {(1, {})}, (1, {1}): {(1, {1})}, (x2^-1, {x2^-1}): {(x2, {x2}), (x2^-1, {x2^-1})}, (x1, {x1}): {(x1, {x1}), (x1^-1, {x1^-1})}, (1, {x1^-1}): {(1, {x1}), (1, {x1^-1})}, (x1^-1, {1, x1^-1}): {(x1^-1, {1, x1^-1}), (x1, {1, x1})}, (1, {x2}): {(1, {x2}), (1, {x2^-1})}, (x2, {x2, 1}): {(x2^-1, {1, x2^-1}), (x2, {x2, 1})}, (x1, {x1^-1, x1}): {(x1^-1, {x1^-1, x1}), (x1, {x1^-1, x1})}, (1, {1, x1^-1}): {(1, {1, x1}), (1, {1, x1^-1})}, (x2^-1, {x2, x2^-1}): {(x2^-1, {x2, x2^-1}), (x2, {x2, x2^-1})}, (1, {x2, 1}): {(1, {x2, 1}), (1, {1, x2^-1})}, (1, {x1^-1, x1}): {(1, {x1^-1, x1})}, (x1, {1, x1^-1, x1}): {(x1^-1, {1, x1^-1, x1}), (x1, {1, x1^-1, x1})}, (1, {x2, x2^-1}): {(1, {x2, x2^-1})}, (x2^-1, {x2, 1, x2^-1}): {(x2^-1, {x2, 1, x2^-1}), (x2, {x2, 1, x2^-1})}, (1, {1, x1^-1, x1}): {(1, {1, x1^-1, x1})}, (1, {x2, 1, x2^-1}): {(1, {x2, 1, x2^-1})}})

In [308]:
# if n_i1 = ... = n_ir, than can replace all the x_{ij} symbols with x_{i1}
eqfact = UF() # which of the n_i's are the same?

nicls = DD(list)
for i,n in enumerate(nis,1):
    nicls[n].append(i)
    
for cl in nicls.values():
    for ind in cl:
        eqfact.add(ind)
        eqfact.union(ind,cl[0])

eqfact.comps()



defaultdict(<class 'set'>, {1: {1, 2}})

In [309]:
def find_min_index(A):
    for i,v in enumerate(xs,1):
        if subset_of_cyclic_factor(A,v):
            return i

hist = set()
mis ={pair: find_min_index(Set([pair[0]]).union(pair[1])) for pair in var_space}
for pair in var_space:
    g,X = pair
    i = mis[pair]
    for h,Y in hist:
        hpair = (h,Y)
        j = mis[hpair]
        # check if n_i==n_j
        if nis[i-1]!=nis[j-1]:
            continue
        h = G(Fk(h).subs({Fk(xs[j-1]):Fk(xs[i-1])}))
        h = rd(h)
        if h!=g:
            continue
        Y = Set([ G(Fk(el).subs({Fk(xs[j-1]):Fk(xs[i-1])})) for el in Y ])
        Y = rd(Y)
        if X==Y:
            eqcl.union(pair,hpair)
            break
    hist.add(pair)

In [310]:
from pprint import pprint
pprint(eqcl.comps())

defaultdict(<class 'set'>,
            {(1, {1, x1^-1, x1}): {(1, {1, x1^-1, x1}), (1, {x2, 1, x2^-1})},
             (1, {x1^-1, x1}): {(1, {x2, x2^-1}), (1, {x1^-1, x1})},
             (1, {1, x1^-1}): {(1, {1, x1^-1}),
                               (1, {1, x2^-1}),
                               (1, {1, x1}),
                               (1, {x2, 1})},
             (1, {x1^-1}): {(1, {x1}), (1, {x2}), (1, {x1^-1}), (1, {x2^-1})},
             (1, {1}): {(1, {1})},
             (1, {}): {(1, {})},
             (x1, {1, x1^-1, x1}): {(x1, {1, x1^-1, x1}),
                                    (x2, {x2, 1, x2^-1}),
                                    (x1^-1, {1, x1^-1, x1}),
                                    (x2^-1, {x2, 1, x2^-1})},
             (x1, {x1^-1, x1}): {(x1, {x1^-1, x1}),
                                 (x2, {x2, x2^-1}),
                                 (x1^-1, {x1^-1, x1}),
                                 (x2^-1, {x2, x2^-1})},
             (x1^-1, {1, x1^-1}): {(x1

In [311]:
# now make subtitutions to the system based on the symetries
reduced_system = []
rev_var_space = {j:i for i,j in var_space.items()}
for eq in system:
    v = eq.lhs()
    pv = rev_var_space[v]
    if pv!=eqcl.find(pv):
        continue
    ag = eq.rhs().args()
    ag = set(ag)-{t}
    ag = list(ag)
    agrepl = [rev_var_space[u] for u in ag]
    agrepl = [eqcl.find(u) for u in agrepl]
    agrepl = [var_space[u] for u in agrepl]
    reduced_system.append(v==(eq.rhs().subs(dict(zip(ag,agrepl)) )))
    

In [313]:
for eq in reduced_system:
    pretty_print(eq)


v0 == v0*(v1 - 1) + 1

v1 == 4*t*v2 + 1

v2 == v6*v7

v7 == t*v14 + t

v6 == (v15 - 1)*v6 + 1

v14 == v22*v23

v15 == t*v14 + 2*t*v2 + 1

v22 == v22*(v28 - 1) + 1

v23 == t

v28 == 2*t*v2 + 1

In [314]:
# get rid of all F-varibles that only depend on t
terminals = {}
for eq in reduced_system:
    if eq.rhs().args()==(t,):
        terminals[eq.lhs()] = eq.rhs()

reduced_system = [eq.subs(terminals) for eq in reduced_system if eq.lhs() not in terminals]


In [319]:
for eq in reduced_system:
    pretty_print(eq)

v0 == v0*(v1 - 1) + 1

v1 == 4*t*v2 + 1

v2 == v6*v7

v7 == t*v14 + t

v6 == (v15 - 1)*v6 + 1

v14 == t*v22

v15 == t*v14 + 2*t*v2 + 1

v22 == v22*(v28 - 1) + 1

v28 == 2*t*v2 + 1

In [320]:
# Take the reduced implicit system of equations, with the list of variables to eliminate,
#  and dump them to a maple data file
import os

var_rem = {eq.lhs() for eq in reduced_system}
print(os.getcwd())
fdir = 'maple_data_files'
if not os.path.isdir(fdir):
    os.mkdir(fdir)
fsuffix = '-dat.maple'
fname = ''
for n in nis:
    fname+='Z%d'%n
if inv_closed:
    fname+='ic'
fname+=fsuffix
fname = os.path.join(fdir,fname)
if not os.path.exists(fname):
    fd = open(fname, 'w')
    fd.write('syst:='+str(reduced_system).replace('==','=')+';')
    fd.write('\n')
    fd.write('varlist:='+str((sorted(var_rem, key = lambda vv: int(str(vv)[1:]))))+';')
    fd.write('\n')
    fd.close()

/home/sage/OneDrive - sfu.ca/Research/Msc_math_sfu/Marni/cogrowth_computations


In [323]:
pretty_print(v0)
v1

v0

v1